In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import concurrent.futures as cf
import re
import time
import random
import creds

Creds

In [3]:
USER = creds.USER
PWD = creds.PWD
URL = "http://20.234.113.211:8100/"

Login

In [56]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(URL)
driver.find_element(By.ID, "dnn_dnnLogin_enhancedLoginLink").click()
driver.switch_to.frame("iPopUp")
user = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "dnn_ctr_Login_Login_DNN_txtUsername"))
user.send_keys(USER)
pwd = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "dnn_ctr_Login_Login_DNN_txtPassword"))
pwd.send_keys(PWD)
WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "dnn_ctr_Login_Login_DNN_cmdLogin")).click()
driver.get("http://20.234.113.211:8100/DesktopModules/Hotcakes/Core/Admin/catalog/default.aspx")

Delete all

In [54]:
def delete_all():
    while True:
        try:
            delete_icon = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.CLASS_NAME, "hcIconDelete"))
            delete_icon.click()
            delete_div = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.CSS_SELECTOR, "ul.hcActions"))
            delete_btn = delete_div.find_element(By.CSS_SELECTOR, "li:first-of-type")
            driver.execute_script('document.querySelector("ul.hcActions li:first-of-type a").click()')
        except Exception as ex:
            break

In [ ]:
question = input("Szeretnéd az összes terméket törölni? [y/n]")

if question == "y":
    delete_all()
else:
    pass

Commit all

In [ ]:
page_url = driver.current_url

termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")
for i in range(len(termekek)):
    try:
        prod_url = driver.current_url
        termekek[i].click()
        save = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "MainContent_btnSave"))
        save.click()
        print(f"{i+1}. termék mentve")
    except:
        input()
    if driver.current_url == prod_url:
        print(prod_url)
        input()
    if driver.current_url != page_url:
        driver.get(page_url)
    termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")

Méret menű hozzáadása

In [42]:
def check_table_for_size():
    """csekkolja, hogy az adott terméknek van-e már méret tulajdonsága"""
    van_tablazat = False
    try:
        driver.find_element(By.CLASS_NAME, "hcGrid")
        van_tablazat = True
    except:
        pass

    van_méret = False

    if van_tablazat:
        trs = driver.find_elements(By.CSS_SELECTOR, "table.formtable.hcGrid")
        for tr in trs:
            kategória_név = tr.find_element(By.CSS_SELECTOR, "td:nth-of-type(2) a").text
            if kategória_név == "Méret":
                van_méret = True
                break

    if van_méret:
        return True
    else:
        return False
    
check_table_for_size()

False

In [43]:
page_url = driver.current_url
termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")


for i in range(len(termekek)):
    termekek[i].click()
    driver.find_element(By.ID, "NavContent_ProductNavigator_hypCustomerChoices").click()

    if check_table_for_size():
        driver.find_element("id", "NavContent_ProductNavigator_hypClose").click()
        if driver.current_url != page_url:
            driver.get(page_url)
        termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")
        continue

    driver.find_element(By.ID, "NavContent_NewChoiceButton").click()
    name_input = driver.find_element(By.ID, "MainContent_NameField")
    name_input.clear()
    name_input.send_keys("Méret")

    sizes = [30, 32, 34, 36, 38, 40] #["XS", "S", "M", "L", "XL"]

    for i in range(len(sizes)):
        # choice_input = WebDriverWait(driver, 10).until(lambda d: d.find_element(By.ID, "MainContent_ItemsEditor_txtNewName"))
        # choice_input.send_keys(sizes[i])
        driver.execute_script(f'document.querySelector("#MainContent_ItemsEditor_txtNewName").value = "{sizes[i]}"')
        # btn = WebDriverWait(driver, 10).until(lambda d: d.find_element("id", "MainContent_ItemsEditor_btnNew"))
        # btn.click()
        driver.execute_script('document.querySelector("#MainContent_ItemsEditor_btnNew").click()')
        time.sleep(0.3)

    driver.find_element("id", "MainContent_btnSaveAndClose").click()
    driver.find_element("id", "NavContent_ProductNavigator_hypClose").click()

    if driver.current_url != page_url:
        driver.get(page_url)
    termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")

In [38]:
férfi_méret = [28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 42]
női_méret = [30, 32, 34, 36, 38, 40]

Kapcsolódó termékek hozzáadása

In [67]:
page_url = driver.current_url
termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")

for i in range(len(termekek)):
    termekek[i].click()

    #region: sku meghatározása
    sku_raw = driver.find_element(By.ID, "MainContent_SkuField").get_attribute("value")
    sku = re.search(r"^.*?_[0-9]", sku_raw).group()[:-2]
    #endregion

    driver.find_element(By.ID, "NavContent_ProductNavigator_hypUpSellCrossSell").click() #related menüpontra kattintás

    #region: szűrés sku-ra
    driver.find_element(By.ID, "MainContent_ucProductPicker_FilterField").clear()
    driver.find_element(By.ID, "MainContent_ucProductPicker_FilterField").send_keys(sku) #filter szöveg beírása
    driver.find_element(By.ID, "MainContent_ucProductPicker_btnGo").click() #keresésre rákattintás
    dropdown = driver.find_element(By.ID, "MainContent_ucProductPicker_DropDownList1") #item per page menű kijelölése
    dropdown.find_element(By.CSS_SELECTOR, "option:last-of-type").click() #legnagyobb megjelenítési lehetőség kijelölése
    #endregion

    clothes = driver.find_elements(By.CSS_SELECTOR, "#MainContent_ucProductPicker_GridView1 tr") #ruhák összegyűjtése
    n = len(clothes)
    selected_rows = random.sample(range(1, n), 5) #ruhák kiválasztása

    #region: ruhák kijelölése
    iteration = 0
    added_item = 0

    while added_item <4:
        selected_sku = clothes[selected_rows[iteration]].find_element(By.CSS_SELECTOR, "td:nth-of-type(2)").text
        if selected_sku == sku_raw:
            iteration += 1
            continue
        clothes[selected_rows[iteration]].find_element(By.CSS_SELECTOR, "td:nth-of-type(1) input").click()
        added_item += 1
        iteration += 1
    #endregion

    driver.find_element(By.ID, "MainContent_btnAdd").click() #ruhák hozzáadása
    driver.find_element(By.ID, "NavContent_ucProductNavigator_hypClose").click()

    if driver.current_url != page_url:
        driver.get(page_url)
    termekek = driver.find_elements(By.CSS_SELECTOR, "a.hcIconEdit")